In [1]:
import pandas as pd
import numpy as np
import os
import copy
import matplotlib.pyplot as plt
import seaborn as sns


def heatmap_plot(contingency_table, x_label, y_label, im_path, val_pair):
    plt.figure(figsize=(10, 7))
    sns.heatmap(contingency_table, annot=True, cmap="YlGnBu")    
    
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.text(0.5, 0.9, f"corr: {np.round(val_pair[0], 2)}, P-value: {np.round(val_pair[1], 3)}", 
             horizontalalignment='center', 
             verticalalignment='center',
             fontsize=15)
    
    if not im_path==None:
        plt.savefig(im_path, dpi=300)
        # plt.show()
    
def measure_resi(sharp_dic, df, resi_cols, measured, threshold=0.2):
    df2process = copy.deepcopy(df)
    remain_lst = [ col for col in resi_cols if col != measured ]    
    df2process['avg_others'] = df2process[remain_lst].mean(axis=1)
    
    df2process[measured + '_diff'] = df2process[measured] - df2process['avg_others']
#     df2process[measured + '_diff'] = np.abs(df2process[measured] - df2process['avg_others'])

    filtered_df = df2process[df2process[measured + '_diff'] > threshold]
    filtered_df = filtered_df.sort_values(measured + '_diff', ascending=False)
    
    sharp_dic.update( {(measured + '_diff'):list(filtered_df['city'].values)} )
    return sharp_dic, filtered_df
                        

def sorting_res(df, col_name, remain=12, abs_=True):
    df2process = copy.deepcopy(df)
    
    if abs_:
        df2process['abs'+col_name] = np.abs(df2process[col_name])
        df_sort = df2process.sort_values(by='abs'+col_name, ascending=False)
    else:
        df_sort = df2process.sort_values(by=col_name, ascending=False)
    
    head_df = df_sort.head(remain)
    tail_df = df_sort.tail(remain)
    
    print(head_df)
    print()
    print(tail_df)
    
    return df_sort, list(head_df['city']), list(tail_df['city'])

    

### power supplementary table formatting -1 spearman analysis results transfer

In [ ]:
root_ = os.path.join(os.getcwd(), 'Syn-DiD Model\\results')
corr_df = pd.read_csv(os.path.join(root_, 'power_spearman_full.csv'))
corr_df

In [ ]:
corr_df['combined'] = corr_df['corr'].apply(lambda x: "{:.3f}".format(x)) + " (" + corr_df['sig'].apply(lambda x: "{:.2e}".format(x)) + ")"

# Pivot the dataframe
df_pivot = corr_df.pivot(index='x', columns='y', values='combined')

df_pivot.head()
df_pivot.to_csv(os.path.join(os.getcwd(), 'power_spearman_full.csv'))


### power supplementary table formatting - 2 date parsing

In [21]:
from datetime import datetime, timedelta, date

path = 'D:\OneDrive\Code Warehouse\Syn-DiD Model\\results\\power_resilience.csv'
info_path = 'D:\OneDrive\Code Warehouse\Syn-DiD Model\\data\\covid19_info_power.csv'
rela_path = 'D:\OneDrive\Code Warehouse\Syn-DiD Model\\data\\relative data\\power_syn_testing.csv'

power_df = pd.read_csv(path, index_col=False)
info_df= pd.read_csv(info_path, index_col=False, encoding='gbk')
rela_df = pd.read_csv(rela_path, index_col=False)


In [23]:
def parse_dates(s, year, split_char='至'):
    start_date_str, end_date_str = s.split(split_char)
    start_date_str = start_date_str.replace('月', '/').replace('日', '')
    end_date_str = end_date_str.replace('月', '/').replace('日', '')
    
    start_date_str = year + '-' + start_date_str
    end_date_str = year + '-' + end_date_str
    return start_date_str, end_date_str

start_dates = []
end_dates = []
nics = []

for i, row in power_df.iterrows():
    city_ = row['city']
    date_ = str(info_df[info_df['city']==city_]['date'].values[0])
    year_ = str(info_df[info_df['city']==city_]['year'].values[0])
    nic_ = rela_df[rela_df['city']==city_]['num_positive'].values[0]
    
    start_date_str, end_date_str = parse_dates(date_, year_)
    start_dates.append(start_date_str)
    end_dates.append(end_date_str)
    nics.append(int(nic_))

    

In [26]:
power_df['start_date'] = start_dates
power_df['end_date'] = end_dates
power_df['nic'] = nics
power_df

,city,state,type,att,att_in,att_pos,resi_all_abs,resi_in_abs,resi_after_abs,extreme,start_date,end_date,nic
0,Anhui,Anhui,2,1.743,0.638,3.992,0.901,0.963,0.371,0.804,2020-1/22,2020-3/8,986.0
1,Beijing,Beijing,2,0.020,0.021,0.019,0.986,0.988,0.971,0.992,2022-1/15,2022-2/6,118.0
2,Beijing 1,Beijing,3,-1.498,-1.787,-0.905,0.484,0.261,0.521,0.465,2022-3/31,2022-6/1,2000.0
3,Beijing 2,Beijing,4,0.667,0.669,0.662,0.410,0.389,0.433,0.488,2020-6/11,2020-7/20,335.0
4,Beijing 3,Beijing,1,-0.687,-0.897,-0.261,0.783,0.719,0.655,0.291,2020-1/31,2020-6/8,420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Xian,Shaanxi,2,-0.013,-0.013,-0.013,0.962,0.910,0.975,0.981,2022-3/5,2022-3/18,387.0
63,Xining,Qinghai,2,-0.273,0.001,-0.775,0.776,0.997,0.364,0.186,2022-5/6,2022-5/28,172.0
64,Xinjiang,Xinjiang,1,-2.936,-4.536,0.165,0.629,0.536,0.574,0.670,2022-7/30,2022-8/30,5460.0
65,Zhejiang,Zhejiang,4,2.073,1.291,3.703,0.804,0.829,0.388,0.730,2020-1/21,2020-3/22,1217.0


In [28]:
supplementary_table_1 = power_df[['city', 'state', 'start_date', 'end_date', 'nic']]
supplementary_table_1.to_csv(os.path.join(os.getcwd(), 'supple_1.csv'))


### absolute ATT analysis

In [ ]:
resi_cols = ['resi_all_abs','resi_in_abs','resi_after_abs','extreme']
att_abs_cols = ['att', 'att_in', 'att_pos']
add_cols = ['city', 'type']

In [3]:
att_abs = pd.concat([power_df[add_cols], power_df[att_abs_cols]*10000], axis=1)
att_df_sort, att_heads, att_tails = sorting_res(att_abs, 'att', 25)


              city  type      att   att_in  att_pos   absatt
64        Xinjiang     1 -29360.0 -45360.0   1650.0  29360.0
14         Gansu 1     4  26390.0  20520.0  38860.0  26390.0
34         Hunan 1     3 -25090.0 -13660.0 -48940.0  25090.0
45        Shanghai     1 -23190.0 -33420.0  -2690.0  23190.0
66      Zhejiang 1     2 -22760.0  -2300.0 -60770.0  22760.0
65        Zhejiang     4  20730.0  12910.0  37030.0  20730.0
30           Henan     4  20700.0  14430.0  32870.0  20700.0
17     Guangzhou 1     3 -20670.0 -19800.0 -28710.0  20670.0
44        Shandong     2  20550.0  21380.0  19650.0  20550.0
36           Jilin     3 -18420.0 -21520.0 -12320.0  18420.0
0            Anhui     2  17430.0   6380.0  39920.0  17430.0
6        Chengdu 1     3 -15110.0 -13710.0 -18130.0  15110.0
2        Beijing 1     3 -14980.0 -17870.0  -9050.0  14980.0
11          Fujian     3 -13210.0  -4880.0 -30500.0  13210.0
31       Huhehaote     4  13100.0   9530.0  20530.0  13100.0
29  Heilongjiang 2     3

In [4]:
temp = list(np.unique([city.split(' ')[0] for city in list(att_df_sort[att_df_sort['absatt']>10000]['city'].values)]))
len(temp)

18

In [5]:
attin_df_sort, attin_heads, attin_tails = sorting_res(att_abs, 'att_in', 25)

              city  type      att   att_in  att_pos  absatt_in
64        Xinjiang     1 -29360.0 -45360.0   1650.0    45360.0
45        Shanghai     1 -23190.0 -33420.0  -2690.0    33420.0
36           Jilin     3 -18420.0 -21520.0 -12320.0    21520.0
44        Shandong     2  20550.0  21380.0  19650.0    21380.0
14         Gansu 1     4  26390.0  20520.0  38860.0    20520.0
17     Guangzhou 1     3 -20670.0 -19800.0 -28710.0    19800.0
2        Beijing 1     3 -14980.0 -17870.0  -9050.0    17870.0
30           Henan     4  20700.0  14430.0  32870.0    14430.0
6        Chengdu 1     3 -15110.0 -13710.0 -18130.0    13710.0
34         Hunan 1     3 -25090.0 -13660.0 -48940.0    13660.0
65        Zhejiang     4  20730.0  12910.0  37030.0    12910.0
29  Heilongjiang 2     3 -12860.0  -9640.0 -19550.0     9640.0
31       Huhehaote     4  13100.0   9530.0  20530.0     9530.0
56       Tianjin 1     1  -7490.0  -9270.0  -4090.0     9270.0
42         Qinghai     4  11800.0   9220.0  17770.0    

In [6]:
attpos_df_sort, attpos_heads, attpos_tails = sorting_res(att_abs, 'att_pos', 30)

              city  type      att   att_in  att_pos  absatt_pos
66      Zhejiang 1     2 -22760.0  -2300.0 -60770.0     60770.0
34         Hunan 1     3 -25090.0 -13660.0 -48940.0     48940.0
0            Anhui     2  17430.0   6380.0  39920.0     39920.0
14         Gansu 1     4  26390.0  20520.0  38860.0     38860.0
53          Suzhou     1  11060.0  -2640.0  38590.0     38590.0
65        Zhejiang     4  20730.0  12910.0  37030.0     37030.0
30           Henan     4  20700.0  14430.0  32870.0     32870.0
11          Fujian     3 -13210.0  -4880.0 -30500.0     30500.0
17     Guangzhou 1     3 -20670.0 -19800.0 -28710.0     28710.0
20         Guizhou     2  12670.0   4690.0  28590.0     28590.0
37         Jilin 1     2  -8860.0     70.0 -27420.0     27420.0
35         Jiangsu     2   8750.0   2010.0  21960.0     21960.0
31       Huhehaote     4  13100.0   9530.0  20530.0     20530.0
49      Shenzhen 1     1  -5510.0   1900.0 -20520.0     20520.0
33           Hunan     2   5990.0  -1580

In [7]:
temp = list(np.unique([city.split(' ')[0] for city in list(attpos_df_sort[attpos_df_sort['absatt_pos']>20000]['city'].values)]))
len(temp)

13

### region-wise CFCR results

In [9]:
path = 'D:\OneDrive\Code Warehouse\Syn-DiD Model\\results\power_radar.csv'
power_df = pd.read_csv(path, index_col=False)

resi_cols = ['resi_all_abs','resi_in_abs','resi_after_abs','extreme']
power_df = power_df[['city'] + resi_cols]


power_df['std_dev'] = power_df[resi_cols].std(axis=1)
sharp_dic = {}


In [12]:
power_df['inte_resi'] = (power_df['resi_all_abs'] + power_df['extreme'])/2
power_df
len(power_df)

41

In [15]:
condi_1 = resi_df_sort['resi_all_abs']>0.9
condi_2 = resi_df_sort['resi_in_abs']>0.9
condi_3 = resi_df_sort['resi_after_abs']>0.8
condi_4 = resi_df_sort['extreme']>0.8
condi_5 = resi_df_sort['inte_resi']>0.9

condi_6 = resi_df_sort['resi_all_abs']<0.5
condi_7 = resi_df_sort['resi_in_abs']<0.5
condi_8 = resi_df_sort['resi_after_abs']<0.5
condi_9 = resi_df_sort['extreme']<0.5
condi_10 = resi_df_sort['inte_resi']<0.5

In [16]:
# large integrated resilience
temp_df = resi_df_sort[condi_5]
temp_df.sort_values('inte_resi', ascending=False)


,city,resi_all_abs,resi_in_abs,resi_after_abs,extreme,std_dev,inte_resi
28,Shaanxi,0.993,0.9900,0.9930,0.992,0.001414,0.9925
37,Xian,0.962,0.9100,0.9750,0.981,0.032321,0.9715
14,Hebei,0.940,0.9350,0.9240,0.942,0.008057,0.9410
26,Shandong,0.942,0.9410,0.7860,0.886,0.073327,0.9140
13,Hangzhou,0.921,0.9555,0.8115,0.884,0.061681,0.9025


In [17]:
# small integrated resilience
temp_df = resi_df_sort[condi_10]
temp_df.sort_values('inte_resi', ascending=False)


,city,resi_all_abs,resi_in_abs,resi_after_abs,extreme,std_dev,inte_resi
38,Xining,0.776,0.997,0.364,0.186,0.371571,0.4810
11,Hainan,0.468,0.384,0.990,0.368,0.294946,0.4180
21,Jinan,0.499,0.545,0.205,0.304,0.160743,0.4015
9,Guiyang,0.223,0.148,0.084,0.359,0.118210,0.2910
24,Qinghai,0.143,0.140,0.116,0.092,0.023796,0.1175


In [18]:
# large all resilience
temp_df = resi_df_sort[condi_1]
temp_df.sort_values('resi_all_abs', ascending=False)

,city,resi_all_abs,resi_in_abs,resi_after_abs,extreme,std_dev,inte_resi
28,Shaanxi,0.993,0.9900,0.9930,0.992,0.001414,0.9925
37,Xian,0.962,0.9100,0.9750,0.981,0.032321,0.9715
19,Jiangsu,0.943,0.9840,0.5270,0.780,0.207320,0.8615
26,Shandong,0.942,0.9410,0.7860,0.886,0.073327,0.9140
14,Hebei,0.940,0.9350,0.9240,0.942,0.008057,0.9410
31,Sichuan,0.925,0.9360,0.8780,0.588,0.164435,0.7565
13,Hangzhou,0.921,0.9555,0.8115,0.884,0.061681,0.9025
25,Qingdao,0.918,0.8060,0.7920,0.414,0.219693,0.6660
0,Anhui,0.901,0.9630,0.3710,0.804,0.267299,0.8525


In [19]:
# small all resilience
resi_df_sort[condi_6]

,city,resi_all_abs,resi_in_abs,resi_after_abs,extreme,std_dev,inte_resi
11,Hainan,0.468,0.384,0.990,0.368,0.294946,0.4180
21,Jinan,0.499,0.545,0.205,0.304,0.160743,0.4015
9,Guiyang,0.223,0.148,0.084,0.359,0.118210,0.2910
24,Qinghai,0.143,0.140,0.116,0.092,0.023796,0.1175


In [20]:
# large in resilience
resi_df_sort[condi_2]

,city,resi_all_abs,resi_in_abs,resi_after_abs,extreme,std_dev,inte_resi
28,Shaanxi,0.99300,0.99000,0.99300,0.9920,0.001414,0.992500
37,Xian,0.96200,0.91000,0.97500,0.9810,0.032321,0.971500
14,Hebei,0.94000,0.93500,0.92400,0.9420,0.008057,0.941000
26,Shandong,0.94200,0.94100,0.78600,0.8860,0.073327,0.914000
13,Hangzhou,0.92100,0.95550,0.81150,0.8840,0.061681,0.902500
19,Jiangsu,0.94300,0.98400,0.52700,0.7800,0.207320,0.861500
34,Tianjin,0.88125,0.91125,0.66275,0.8240,0.110787,0.852625
0,Anhui,0.90100,0.96300,0.37100,0.8040,0.267299,0.852500
7,Guangxi,0.88600,0.94800,0.42400,0.7060,0.234938,0.796000
10,Guizhou,0.83100,0.91000,0.36600,0.6910,0.240067,0.761000


In [21]:
# small in-resilience 
resi_df_sort[condi_7]

,city,resi_all_abs,resi_in_abs,resi_after_abs,extreme,std_dev,inte_resi
11,Hainan,0.468,0.384,0.990,0.368,0.294946,0.4180
9,Guiyang,0.223,0.148,0.084,0.359,0.118210,0.2910
24,Qinghai,0.143,0.140,0.116,0.092,0.023796,0.1175


In [22]:
# large post resilience (0.8)
resi_df_sort[condi_3]

,city,resi_all_abs,resi_in_abs,resi_after_abs,extreme,std_dev,inte_resi
28,Shaanxi,0.993,0.9900,0.9930,0.992,0.001414,0.9925
37,Xian,0.962,0.9100,0.9750,0.981,0.032321,0.9715
14,Hebei,0.940,0.9350,0.9240,0.942,0.008057,0.9410
13,Hangzhou,0.921,0.9555,0.8115,0.884,0.061681,0.9025
31,Sichuan,0.925,0.9360,0.8780,0.588,0.164435,0.7565
11,Hainan,0.468,0.3840,0.9900,0.368,0.294946,0.4180


In [23]:
# small pos-resilience 
temp_df = resi_df_sort[condi_8]
temp_df.sort_values('resi_after_abs', ascending=False)

,city,resi_all_abs,resi_in_abs,resi_after_abs,extreme,std_dev,inte_resi
23,Nanjing,0.838000,0.8495,0.4545,0.767000,0.185457,0.802500
15,Heilongjiang,0.715333,0.8350,0.4340,0.514333,0.183474,0.614833
7,Guangxi,0.886000,0.9480,0.4240,0.706000,0.234938,0.796000
29,Shenzhen,0.697500,0.7970,0.4100,0.551500,0.169296,0.624500
12,Haerbin,0.715000,0.6785,0.4035,0.708000,0.149340,0.711500
22,Nanchang,0.620000,0.6420,0.3730,0.583000,0.123425,0.601500
0,Anhui,0.901000,0.9630,0.3710,0.804000,0.267299,0.852500
3,Chongqing,0.821000,0.8465,0.3670,0.746500,0.222908,0.783750
40,Zhejiang,0.796000,0.8650,0.3660,0.563500,0.227784,0.679750
10,Guizhou,0.831000,0.9100,0.3660,0.691000,0.240067,0.761000


In [24]:
# large extreme
resi_df_sort[condi_4]

,city,resi_all_abs,resi_in_abs,resi_after_abs,extreme,std_dev,inte_resi
28,Shaanxi,0.99300,0.99000,0.99300,0.9920,0.001414,0.992500
37,Xian,0.96200,0.91000,0.97500,0.9810,0.032321,0.971500
14,Hebei,0.94000,0.93500,0.92400,0.9420,0.008057,0.941000
26,Shandong,0.94200,0.94100,0.78600,0.8860,0.073327,0.914000
13,Hangzhou,0.92100,0.95550,0.81150,0.8840,0.061681,0.902500
34,Tianjin,0.88125,0.91125,0.66275,0.8240,0.110787,0.852625
0,Anhui,0.90100,0.96300,0.37100,0.8040,0.267299,0.852500
35,Wuhan,0.84950,0.86700,0.66350,0.8045,0.092253,0.827000


In [25]:
# small extreme
temp_df = resi_df_sort[condi_9]
temp_df.sort_values('extreme', ascending=False)

,city,resi_all_abs,resi_in_abs,resi_after_abs,extreme,std_dev,inte_resi
20,Jilin,0.5790,0.699000,0.297500,0.480500,0.169749,0.529750
8,Guangzhou,0.6670,0.618333,0.522667,0.457667,0.094080,0.562333
27,Shanghai,0.7050,0.616500,0.769000,0.451000,0.137905,0.578000
25,Qingdao,0.9180,0.806000,0.792000,0.414000,0.219693,0.666000
11,Hainan,0.4680,0.384000,0.990000,0.368000,0.294946,0.418000
9,Guiyang,0.2230,0.148000,0.084000,0.359000,0.118210,0.291000
21,Jinan,0.4990,0.545000,0.205000,0.304000,0.160743,0.401500
18,Hunan,0.7175,0.861000,0.318500,0.290500,0.286165,0.504000
38,Xining,0.7760,0.997000,0.364000,0.186000,0.371571,0.481000
33,Taiyuan,0.8430,0.972000,0.556000,0.181000,0.350787,0.512000


### Region-wise CFCR analysis (power)

In [26]:
threshold = 0.15

filtered_cities = power_df[power_df['std_dev'] > threshold]
filtered_cities


,city,resi_all_abs,resi_in_abs,resi_after_abs,extreme,std_dev,inte_resi
0,Anhui,0.901000,0.9630,0.3710,0.804000,0.267299,0.852500
3,Chongqing,0.821000,0.8465,0.3670,0.746500,0.222908,0.783750
5,Fujian,0.847500,0.9100,0.5520,0.650500,0.167143,0.749000
7,Guangxi,0.886000,0.9480,0.4240,0.706000,0.234938,0.796000
10,Guizhou,0.831000,0.9100,0.3660,0.691000,0.240067,0.761000
11,Hainan,0.468000,0.3840,0.9900,0.368000,0.294946,0.418000
15,Heilongjiang,0.715333,0.8350,0.4340,0.514333,0.183474,0.614833
18,Hunan,0.717500,0.8610,0.3185,0.290500,0.286165,0.504000
19,Jiangsu,0.943000,0.9840,0.5270,0.780000,0.207320,0.861500
20,Jilin,0.579000,0.6990,0.2975,0.480500,0.169749,0.529750


In [32]:
# sharp at in resilience
sharp_dic, filtered_df = measure_resi(sharp_dic, power_df, resi_cols, 'inte_resi', 0.05)
filtered_df

,city,resi_all_abs,resi_in_abs,resi_after_abs,extreme,std_dev,inte_resi,avg_others,inte_resi_diff
0,Anhui,0.9010,0.9630,0.3710,0.8040,0.267299,0.85250,0.75975,0.09275
3,Chongqing,0.8210,0.8465,0.3670,0.7465,0.222908,0.78375,0.69525,0.08850
9,Guiyang,0.2230,0.1480,0.0840,0.3590,0.118210,0.29100,0.20350,0.08750
12,Haerbin,0.7150,0.6785,0.4035,0.7080,0.149340,0.71150,0.62625,0.08525
23,Nanjing,0.8380,0.8495,0.4545,0.7670,0.185457,0.80250,0.72725,0.07525
2,Chengdu,0.6235,0.5230,0.3200,0.5025,0.126422,0.56300,0.49225,0.07075
36,Wulumuqi,0.7480,0.6580,0.5260,0.6980,0.095084,0.72300,0.65750,0.06550
10,Guizhou,0.8310,0.9100,0.3660,0.6910,0.240067,0.76100,0.69950,0.06150
7,Guangxi,0.8860,0.9480,0.4240,0.7060,0.234938,0.79600,0.74100,0.05500
17,Huhehaote,0.6175,0.6325,0.3235,0.5505,0.142853,0.58400,0.53100,0.05300


In [27]:
# sharp at all resilience
sharp_dic, filtered_df = measure_resi(sharp_dic, power_df, resi_cols, 'resi_all_abs', 0.15)

out_dir = 'D:\\power_strength_all.csv'
filtered_df = np.round(filtered_df, 3)
# filtered_df.to_csv(out_dir, index=False)
filtered_df


,city,resi_all_abs,resi_in_abs,resi_after_abs,extreme,std_dev,inte_resi,avg_others,resi_all_abs_diff
33,Taiyuan,0.843,0.972,0.556,0.181,0.351,0.512,0.570,0.273
38,Xining,0.776,0.997,0.364,0.186,0.372,0.481,0.516,0.260
25,Qingdao,0.918,0.806,0.792,0.414,0.220,0.666,0.671,0.247
18,Hunan,0.718,0.861,0.318,0.290,0.286,0.504,0.490,0.228
40,Zhejiang,0.796,0.865,0.366,0.564,0.228,0.680,0.598,0.198
7,Guangxi,0.886,0.948,0.424,0.706,0.235,0.796,0.693,0.193
0,Anhui,0.901,0.963,0.371,0.804,0.267,0.852,0.713,0.188
19,Jiangsu,0.943,0.984,0.527,0.780,0.207,0.861,0.764,0.179
10,Guizhou,0.831,0.910,0.366,0.691,0.240,0.761,0.656,0.175
2,Chengdu,0.624,0.523,0.320,0.502,0.126,0.563,0.448,0.175


In [28]:
# sharp at in resilience
sharp_dic, filtered_df = measure_resi(sharp_dic, power_df, resi_cols, 'resi_in_abs', 0.15)

out_dir = 'D:\\power_strength_in.csv'
filtered_df = np.round(filtered_df, 3)
# filtered_df.to_csv(out_dir, index=False)
filtered_df


,city,resi_all_abs,resi_in_abs,resi_after_abs,extreme,std_dev,inte_resi,avg_others,resi_in_abs_diff
38,Xining,0.776,0.997,0.364,0.186,0.372,0.581,0.442,0.555
33,Taiyuan,0.843,0.972,0.556,0.181,0.351,0.638,0.527,0.445
18,Hunan,0.718,0.861,0.318,0.290,0.286,0.547,0.442,0.419
40,Zhejiang,0.796,0.865,0.366,0.564,0.228,0.648,0.575,0.290
10,Guizhou,0.831,0.910,0.366,0.691,0.240,0.700,0.629,0.281
15,Heilongjiang,0.715,0.835,0.434,0.514,0.183,0.625,0.555,0.280
7,Guangxi,0.886,0.948,0.424,0.706,0.235,0.741,0.672,0.276
0,Anhui,0.901,0.963,0.371,0.804,0.267,0.760,0.692,0.271
20,Jilin,0.579,0.699,0.298,0.480,0.170,0.514,0.452,0.247
29,Shenzhen,0.698,0.797,0.410,0.552,0.169,0.614,0.553,0.244


In [29]:
# sharp at post resilience
sharp_dic, filtered_df = measure_resi(sharp_dic, power_df, resi_cols, 'resi_after_abs', 0.1)

out_dir = 'D:\\power_strength_pos.csv'
filtered_df = np.round(filtered_df, 3)
filtered_df.to_csv(out_dir, index=False)
filtered_df

,city,resi_all_abs,resi_in_abs,resi_after_abs,extreme,std_dev,inte_resi,avg_others,resi_after_abs_diff
11,Hainan,0.468,0.384,0.990,0.368,0.295,0.552,0.407,0.583
27,Shanghai,0.705,0.616,0.769,0.451,0.138,0.635,0.591,0.178


In [30]:
# sharp at extreme resilience
sharp_dic, filtered_df = measure_resi(sharp_dic, power_df, resi_cols, 'extreme', 0.15)

out_dir = 'D:\\power_strength_redundacy.csv'
filtered_df = np.round(filtered_df, 3)
filtered_df.to_csv(out_dir, index=False)
filtered_df


,city,resi_all_abs,resi_in_abs,resi_after_abs,extreme,std_dev,inte_resi,avg_others,extreme_diff
9,Guiyang,0.223,0.148,0.084,0.359,0.118,0.204,0.152,0.207
